In [2]:
import pandas as pd # library for data analsysis
import numpy as np

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [8]:
r = requests.get('https://usc.data.socrata.com/api/geospatial/r8qd-yxsr?method=export&format=GeoJSON')
losangeles_data = r.json()

In [9]:
neighborhoods_data = losangeles_data['features']

In [12]:
neighborhoods_data[2]

{'type': 'Feature',
 'properties': {'external_i': 'agoura-hills',
  'name': 'Agoura Hills',
  'location': 'POINT(34.146736499122795 -118.75988450000015)',
  'latitude': '-118.75988450000015',
  'slug_1': None,
  'sqmi': '8.14676029818',
  'display_na': 'Agoura Hills L.A. County Neighborhood (Current)',
  'set': 'L.A. County Neighborhoods (Current)',
  'slug': 'agoura-hills',
  'longitude': '34.146736499122795',
  'name_1': None,
  'kind': 'L.A. County Neighborhood (Current)',
  'type': 'standalone-city'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-118.76192500000009, 34.16820299912263],
     [-118.72631699999998, 34.16787499912258],
     [-118.73232400000018, 34.1543019991227],
     [-118.73706000000027, 34.15435299912277],
     [-118.73702385723149, 34.14304324369543],
     [-118.73587193294368, 34.14267020794711],
     [-118.73493704578691, 34.14215345623102],
     [-118.7306383000593, 34.139239215941366],
     [-118.72918210881832, 34.13843288950915],
     [-118.727

In [13]:
neighborhoods = pd.DataFrame(columns=['Neighborhood', 'Latitude', 'Longitude'] )
neighborhoods

,Neighborhood,Latitude,Longitude


In [14]:
for data in neighborhoods_data:
    neighborhood_name = data['properties']['name']
    neighborhood_lat = data['properties']['latitude']
    neighborhood_lon = data['properties']['longitude']
    
    neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [16]:
neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Acton,-118.16981019229348,34.497355239240846
1,Adams-Normandie,-118.30020800000011,34.031461499124156
2,Agoura Hills,-118.75988450000015,34.146736499122795
3,Agua Dulce,-118.3171036690717,34.504926999796837
4,Alhambra,-118.13651200000021,34.085538999123571


In [17]:
address = 'Los Angeles, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles, CA are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Los Angeles, CA are 34.0536909, -118.2427666.


## The values in the JSON file are all around 34, -118. However, latitude and longitude are reversed, so we will change the column names to reflect this.

In [31]:
neighborhoods.rename(columns = {'Latitude':'Longitude', 'Longitude':'Latitude'}, inplace=True)
neighborhoods.head()

,Neighborhood,Longitude,Latitude
0,Acton,-118.16981019229348,34.497355239240846
1,Adams-Normandie,-118.30020800000011,34.031461499124156
2,Agoura Hills,-118.75988450000015,34.146736499122795
3,Agua Dulce,-118.3171036690717,34.504926999796837
4,Alhambra,-118.13651200000021,34.085538999123571


In [32]:
neighborhoods.shape

(272, 3)

In [33]:
neighborhoods.dropna(subset=['Latitude'], inplace=True)
neighborhoods.dropna(subset=['Longitude'], inplace=True)
neighborhoods.shape

(272, 3)

In [24]:
map_los_angeles = folium.Map(location=[latitude, longitude], zoom_start=10)
map_los_angeles

In [34]:
for lat, lng, in zip(neighborhoods['Latitude'], neighborhoods['Longitude']):
    map_los_angeles.add_child(
        folium.features.CircleMarker(
            [float(lat), float(lng)],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )
    
map_los_angeles

# Foursquare API

In [35]:
CLIENT_ID = 'ERFCS0HI2ODMI5PVVGK2FXA0BBDO5245K3YHLDTXF4UBWPBG' # your Foursquare ID
CLIENT_SECRET = 'RP2Z2TV0DWKA10HHRQI5YYI5LCVBYTWTQHNMQ41PXAN2BRM0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ERFCS0HI2ODMI5PVVGK2FXA0BBDO5245K3YHLDTXF4UBWPBG
CLIENT_SECRET:RP2Z2TV0DWKA10HHRQI5YYI5LCVBYTWTQHNMQ41PXAN2BRM0


In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
LIMIT=100

los_angeles_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Acton
Adams-Normandie
Agoura Hills
Agua Dulce
Alhambra
Alondra Park
Artesia
Altadena
Angeles Crest
Arcadia
Arleta
Arlington Heights
Athens
Atwater Village
Avalon
Avocado Heights
Azusa
Vermont-Slauson
Baldwin Hills/Crenshaw
Baldwin Park
Bel-Air
Bellflower
Bell Gardens
Green Valley
Bell
Beverly Crest
Beverly Grove
Burbank
Koreatown
Beverly Hills
Beverlywood
Boyle Heights
Bradbury
Brentwood
Broadway-Manchester
Calabasas
Canoga Park
Carson
Carthay
Castaic Canyons
Chatsworth
Castaic
Central-Alameda
Century City
Cerritos
Charter Oak
Chatsworth Reservoir
Chesterfield Square
Cheviot Hills
Chinatown
Citrus
Claremont
Northridge
Commerce
Compton
Cypress Park
La Mirada
Covina
Cudahy
Culver City
Del Aire
Del Rey
Desert View Highlands
Diamond Bar
Downey
Downtown
Duarte
Eagle Rock
East Compton
East Hollywood
East La Mirada
Elizabeth Lake
East Los Angeles
East Pasadena
East San Gabriel
Echo Park
El Monte
El Segundo
El Sereno
Elysian Park
Elysian Valley
Vermont Square
Encino
Exposition Park
Fairfax
Flo

In [39]:
print(los_angeles_venues.shape)
los_angeles_venues.head()

(2989, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton,34.497355239240846,-118.16981019229348,Epik Engineering,34.498718,-118.168046,Construction & Landscaping
1,Acton,34.497355239240846,-118.16981019229348,Alma Gardening Co.,34.494762,-118.172550,Construction & Landscaping
2,Adams-Normandie,34.031461499124156,-118.30020800000011,Orange Door Sushi,34.032485,-118.299368,Sushi Restaurant
3,Adams-Normandie,34.031461499124156,-118.30020800000011,Shell,34.033095,-118.300025,Gas Station
4,Adams-Normandie,34.031461499124156,-118.30020800000011,Sushi Delight,34.032445,-118.299525,Sushi Restaurant


In [40]:
los_angeles_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acton,2,2,2,2,2,2
Adams-Normandie,9,9,9,9,9,9
Agoura Hills,30,30,30,30,30,30
Agua Dulce,1,1,1,1,1,1
Alhambra,13,13,13,13,13,13
Alondra Park,4,4,4,4,4,4
Altadena,5,5,5,5,5,5
Arcadia,21,21,21,21,21,21
Arleta,4,4,4,4,4,4


In [41]:
print('There are {} uniques categories.'.format(len(los_angeles_venues['Venue Category'].unique())))

There are 322 uniques categories.


# Analyzing the Data

#### Frequency of each venue category.
We use one-hot encoding for this.

In [42]:
los_angeles_onehot = pd.get_dummies(los_angeles_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
los_angeles_onehot['Neighborhood'] = los_angeles_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [los_angeles_onehot.columns[-1]] + list(los_angeles_onehot.columns[:-1])
los_angeles_onehot = los_angeles_onehot[fixed_columns]

los_angeles_onehot.head()

,Yoga Studio,ATM,Accessories Store,Airport,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Warehouse Store,Watch Shop,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
los_angeles_onehot.shape

(2989, 322)

#### Grouping by neighborhood.
We calculate the mean of each frequency.

In [45]:
los_angeles_grouped = los_angeles_onehot.groupby('Neighborhood').mean().reset_index()
los_angeles_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Airport,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Warehouse Store,Watch Shop,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Acton,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Adams-Normandie,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Agoura Hills,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Agua Dulce,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alhambra,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
los_angeles_grouped.shape

(241, 322)